In [1]:
def preprocess(test_data):
    import pandas as pd
    import numpy as np
    from sklearn import preprocessing
    from sklearn.preprocessing import MinMaxScaler
    
    test_data_sid = test_data.sid.values

    test_data = test_data.drop(columns= ['sid','pid','plans','req_time','plan_time','o','d'])
    #one hot 'time_hour'
    df2 = pd.get_dummies(test_data['time_hour'])
    test_data = test_data.join(df2)
    
    test_data = test_data.drop(columns = ['time_hour'])

    scaler = MinMaxScaler()
    test_data = scaler.fit_transform(test_data)
    test_data = pd.DataFrame(test_data)
    
    return test_data , test_data_sid

In [2]:
def predict_sid():
    import pandas as pd
    import numpy as np
    import csv
    
    test_merges1 = pd.read_csv('/storage/kdd/test_final.csv')
    total_sid = test_merges1.sid.values
    
    total = pd.DataFrame({'sid': total_sid})
    return total

In [3]:
def load_part_ans():
    import pandas as pd
    import numpy as np
    import csv
    
    part_ans = pd.read_csv('/storage/kdd/part_answer.csv')
    return part_ans

In [4]:
def load_and_train(test_data , test_data_sid):
    import pandas as pd
    import numpy as np
    from sklearn.externals import joblib 
    
    #randomforest model
    rf = joblib.load('rf4.pkl') 
    test_label = rf.predict(test_data)
    
    ans = pd.DataFrame({'sid': test_data_sid, 'recommend_mode':test_label})
    ans = ans[['sid','recommend_mode']]
    
    return ans

In [5]:
def main():
    import pandas as pd
    import numpy as np
    import csv
    
    test_data = pd.read_csv('/storage/kdd/test_final3.csv')#這個檔案就是去除no_plan no_pid剩下的data
    test_data , test_data_sid = preprocess(test_data)
    total = predict_sid()
    part_ans = load_part_ans()
    ans = load_and_train(test_data , test_data_sid)
    
    ans = pd.concat([ans,part_ans], ignore_index= True)
    ans = total.merge(ans,left_on='sid',right_on='sid')
    
    ans.to_csv('rftest.csv' , index = False)

In [6]:
if __name__ == '__main__':
    main()